# Transformers inference acceleration

> 以基于bert的意图识别模型为例

In [9]:
import numpy as np
from pathlib import Path
from transformers import pipeline
from time import perf_counter
from datasets import load_dataset
import evaluate, torch

bert_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
pipe = pipeline("text-classification", model=bert_ckpt)

query = """Hey, I'd like to rent a vehicle from Nov 1st to Nov 15th
in
Paris and I need a 15 passenger van"""
result = pipe(query)
print(result)


class PerformanceBenchmark:

    def __init__(self, pipeline, dataset, optim_type="BERTbaseline"):
        self.pipeline = pipeline
        self.dataset = dataset
        self.optim_type = optim_type

    def compute_accuracy(self):
        preds, labels = [], []
        for example in self.dataset:
            pred = self.pipeline(example["text"])[0]["label"]
        label = example["intent"]
        preds.append(intents.str2int(pred))
        labels.append(label)
        accuracy = accuracy_score.compute(predictions=preds,
                                          references=labels)
        print(f"Accuracy on test set - {accuracy['accuracy']:.3f}")
        return accuracy

    def compute_size(self):
        state_dict = self.pipeline.model.state_dict()
        tmp_path = Path("model.pt")
        torch.save(state_dict, tmp_path)
        # Calculate size in megabytes
        size_mb = Path(tmp_path).stat().st_size / (1024 * 1024)
        # Delete temporary file
        tmp_path.unlink()
        print(f"Model size (MB) - {size_mb:.2f}")
        return {"size_mb": size_mb}

    def time_pipeline(self, query="What is the pin number for myaccount?"):
        latencies = []
        # Warmup
        for _ in range(10):
            _ = self.pipeline(query)
        # Timed run
        for _ in range(100):
            start_time = perf_counter()
        _ = self.pipeline(query)
        latency = perf_counter() - start_time
        latencies.append(latency)
        # Compute run statistics
        time_avg_ms = 1000 * np.mean(latencies)
        time_std_ms = 1000 * np.std(latencies)
        print(f"Average latency (ms) - {time_avg_ms:.2f} +\-{time_std_ms: .2f}")
        return {"time_avg_ms": time_avg_ms, "time_std_ms": time_std_ms}


    def run_benchmark(self):
        metrics = {}
        metrics[self.optim_type] = self.compute_size()
        metrics[self.optim_type].update(self.time_pipeline())
        metrics[self.optim_type].update(self.compute_accuracy())
        return metrics

        


clinc = load_dataset("clinc_oos", "plus")

sample = clinc["test"][42]
print(sample)

intents = clinc["test"].features["intent"]
intents.int2str(sample["intent"])

accuracy_score = evaluate.load("accuracy")
print(accuracy_score)

list(pipe.model.state_dict().items())[42]
# torch.save(pipe.model.state_dict(), "model.pt")

pb = PerformanceBenchmark(pipe, clinc['test'])
perf_metrics = pb.run_benchmark()
print(perf_metrics)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'car_rental', 'score': 0.5490034818649292}]
{'text': 'transfer $100 from my checking to saving account', 'intent': 133}
EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >

In [3]:
!pip install scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 10.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 10.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## Knowledge distillation 知识蒸馏

核心思路是：在真实标签的基础上，再加上一份来自教师模型的“软概率分布”。这份分布可以给学生模型提供额外的信息。

### Knowledge distillation for Fine-Tuning

举个例子，比如我们有个 BERT-base 分类器（老师），它对多个意图都给出了较高的概率。这可能说明这些意图在特征空间中彼此靠得很近。那我们就可以训练学生模型去“模仿”这些概率分布。

通过这种方式，我们希望把老师模型里学到的一些**“暗知识”（dark knowledge）**提炼出来给学生。所谓“暗知识”就是那些单靠真实标签是学不到的信息。
<br/><br/>
<img src="./imgs/student_teacher_soften_max.png" width="400"/>
<br/><br/>

*KL(Kullback–Leibler)* : 衡量学生老师模型分别生成的logits 概率差异，计算公式如下：
<br/><br/>

$D_KL(p, q) = \sum_{i}pi(x)log\frac{p_i(x)}{q_i(x)}$

<br/><br/>可用于定义知识蒸馏的loss <br/><br/>
$L_{KD} = T^2D_{KL}$

在知识蒸馏里我们会用温度 T 来“软化”老师模型的输出概率，但这样会让梯度变小（大概变成原来的 1/T²），所以我们引入一个 T² 的归一化系数来把梯度调回合适的范围。温度 T 越高，
softmax 输出的概率分布就越“平”，反之越尖锐，而这个平滑度直接影响训练时梯度的大小，所以引入 T² 是为了让蒸馏过程中的训练更稳定有效。

## Quantization 量化

## Pruning 剪枝

## Graph optimization 图优化